In [1]:
# Be quiet!!
import sys
import logging
logging.getLogger().setLevel(logging.ERROR)
logging.disable(sys.maxsize)

import copy
import torch
from torch.utils.data import *
from transformers import *
import inspect
sys.path.insert(0, "..")

from models import *
from logic import *
from my_datasets import *

from utils import *
import numpy as np


/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
n, r = 5, 8
ap, bp, tp, sp = 0.2, 0.2, 0.4, 0.1

num_arsteps = 3

train_len = 2500
test_len = 500
num_epochs = 30
test_is_train = True

In [3]:
qed_training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy = "epoch",
    num_train_epochs = num_epochs,
    per_device_train_batch_size = 24,
    per_device_eval_batch_size = 24,
    logging_steps = 5
)

succ_training_args = qed_training_args
arsteps_training_args = qed_training_args

In [4]:
qed_train_dataset_config = OneShotQedDatasetConfig(r,n,ap,bp,tp,dataset_len=train_len,seed=1234)
qed_test_dataset_config = OneShotQedDatasetConfig(r,n,ap,bp,tp,dataset_len=test_len,seed=2345)
qed_train_dataset = OneShotQedDataset(qed_train_dataset_config)
qed_test_dataset = OneShotQedDataset(qed_test_dataset_config)

succ_train_dataset_config = PredictSuccDatasetConfig(r,n,ap,bp,tp,dataset_len=train_len,seed=1234)
succ_test_dataset_config = PredictSuccDatasetConfig(r,n,ap,bp,tp,dataset_len=test_len,seed=2345)
succ_train_dataset = PredictSuccDataset(succ_train_dataset_config)
succ_test_dataset = PredictSuccDataset(succ_test_dataset_config)

arsteps_train_dataset_config = AutoRegFixedStepsDatasetConfig(r,n,ap,bp,sp,num_arsteps,dataset_len=train_len,seed=1234)
arsteps_test_dataset_config = AutoRegFixedStepsDatasetConfig(r,n,ap,bp,sp,num_arsteps,dataset_len=test_len,seed=2345)
arsteps_train_dataset = AutoRegFixedStepsDataset(arsteps_train_dataset_config)
arsteps_test_dataset = AutoRegFixedStepsDataset(arsteps_test_dataset_config)

In [5]:
## Initialize a bunch of models

mytf_qed_base = get_seqcls_model("mytf", num_labels=2, problem_type="single_label_classification")
mytf_qed = OneShotQedTaskModel(OneShotQedTaskConfig(n, mytf_qed_base))

mytf_seqcls_base = get_seqcls_model("mytf", num_labels=n, problem_type="multi_label_classification")
mytf_succ = PredictSuccTaskModel(PredictSuccTaskConfig(n, mytf_seqcls_base))
mytf_arsteps = AutoRegFixedStepsTaskModel(AutoRegFixedStepsTaskConfig(n, num_arsteps, mytf_seqcls_base))

gpt2_qed_base = get_seqcls_model("gpt2", num_labels=2, problem_type="single_label_classification")
gpt2_qed = OneShotQedTaskModel(OneShotQedTaskConfig(n, gpt2_qed_base))

gpt2_seqcls_base = get_seqcls_model("gpt2", num_labels=n, problem_type="multi_label_classification")
gpt2_succ = PredictSuccTaskModel(PredictSuccTaskConfig(n, gpt2_seqcls_base))
gpt2_arsteps = AutoRegFixedStepsTaskModel(AutoRegFixedStepsTaskConfig(n, num_arsteps, gpt2_seqcls_base))

In [6]:
### QED GPT2
gpt2_qed_trainer = Trainer(gpt2_qed, qed_training_args,
    train_dataset = qed_train_dataset,
    eval_dataset = qed_train_dataset if test_is_train else qed_test_dataset,
    compute_metrics = qed_compute_metrics)
gpt2_qed_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.713200,0.762988,0.514000,0.091200
2,0.667300,0.706389,0.682000,0.818400
3,0.463500,0.551684,0.735600,0.647200
4,0.318300,0.609140,0.742400,0.565200
5,0.235200,3.079830,0.563200,0.999600
6,0.097100,2.657287,0.590000,0.972000
7,0.089000,0.930387,0.738000,0.540000
8,0.077300,0.690905,0.744000,0.634000
9,0.006900,1.235873,0.735600,0.509600
10,0.002700,1.042571,0.748000,0.557200


TrainOutput(global_step=3150, training_loss=0.1847692497222821, metrics={'train_runtime': 305.6217, 'train_samples_per_second': 245.401, 'train_steps_per_second': 10.307, 'total_flos': 0.0, 'train_loss': 0.1847692497222821, 'epoch': 30.0})

In [7]:
### QED MyTf
mytf_qed_trainer = Trainer(mytf_qed, qed_training_args,
    train_dataset = qed_train_dataset,
    eval_dataset = qed_train_dataset if test_is_train else qed_test_dataset,
    compute_metrics = qed_compute_metrics)
# mytf_qed_trainer.train()

In [8]:
### SUCC GPT2
gpt2_succ_trainer = Trainer(gpt2_succ, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_train_dataset if test_is_train else succ_test_dataset,
    compute_metrics = succ_compute_metrics)
gpt2_succ_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.325000,0.294123,0.862160,0.790000
2,0.173900,0.200524,0.912640,0.737920
3,0.107400,0.133453,0.945920,0.797760
4,0.048300,0.147108,0.945680,0.804880
5,0.051600,0.118173,0.952880,0.790960
6,0.029200,0.113127,0.957120,0.786400
7,0.029600,0.129322,0.958080,0.759040
8,0.018900,0.134202,0.958240,0.760640
9,0.017800,0.148854,0.952960,0.753440
10,0.008500,0.129315,0.959600,0.782000


TrainOutput(global_step=3150, training_loss=0.04018355790540458, metrics={'train_runtime': 269.7095, 'train_samples_per_second': 278.077, 'train_steps_per_second': 11.679, 'total_flos': 0.0, 'train_loss': 0.04018355790540458, 'epoch': 30.0})

In [9]:
### SUCC MyTF
mytf_succ_trainer = Trainer(mytf_succ, succ_training_args,
    train_dataset = succ_train_dataset,
    eval_dataset = succ_train_dataset if test_is_train else succ_test_dataset,
    compute_metrics = succ_compute_metrics)
# mytf_succ_trainer.train()

In [10]:
### ARSteps GPT2
gpt2_arsteps_trainer = Trainer(gpt2_arsteps, arsteps_training_args,
    train_dataset = arsteps_train_dataset,
    eval_dataset = arsteps_train_dataset if test_is_train else arsteps_test_dataset,
    compute_metrics = arsteps_compute_metrics)
gpt2_arsteps_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Avg ones
1,0.575600,0.571667,0.724907,0.754693
2,0.514500,0.519038,0.819120,0.682133
3,0.475100,0.491305,0.872027,0.670133
4,0.444400,0.460896,0.937013,0.677733
5,0.429200,0.451378,0.948000,0.654640
6,0.426900,0.446925,0.960240,0.670240
7,0.422000,0.458953,0.951680,0.728133
8,0.419500,0.458463,0.952587,0.730693
9,0.418600,0.445872,0.969627,0.702880
10,0.417300,0.443963,0.971147,0.693573


TrainOutput(global_step=3150, training_loss=0.44804335102202403, metrics={'train_runtime': 551.3116, 'train_samples_per_second': 136.039, 'train_steps_per_second': 5.714, 'total_flos': 0.0, 'train_loss': 0.44804335102202403, 'epoch': 30.0})

In [11]:
batch = next(iter(DataLoader(arsteps_train_dataset, batch_size=7)))
gpt2_arsteps.cpu()
out = gpt2_arsteps(**batch)

In [12]:
out.logits.shape

torch.Size([7, 3, 5])

In [13]:
x = np.array([True, False])
y = np.array([1, 0])

In [14]:
x == y

array([ True,  True])

In [15]:
x.astype("long")

array([1, 0])

In [16]:
np.int64

numpy.int64